### 1. Importing the libraries

In [0]:
import pandas as pd
import os
import json
import ast
import string

In [0]:
### Character mapping for the feature renaming
character_mapping = {
 '!':"exclamation",
 '"':"double_quotes",
 '#':"hashtag",
 '$':"dollar",
 '%':"percentage",
 '&':"ampersand",
 "'":"single_quotes",
 '(':"open_round_bracket",
 ')':"close_round_bracket",
 '*':"asterisk",
 '+':"plus",
 ',':"comma",
 '-':"minus",
 '.':"full_stop",
 '/':"forward_slash",
 ':':"colon",
 ';':"semi_colon",
 '<':"less_than",
 '=':"equal_to",
 '>':"greater_than",
 '?':"question_mark",
 '@':"at_the_rate",
 '[':"open_square_bracket",
 ']':"close_square_bracket",
 '_':"underscore",
 '`':"escape_character",
 '{':"open_curly_bracket",
 '|':"vertical_bar",
 '}':"close_curly_bracket",
 '~':"tilde",
 ' ':"space",
 '\x0c':"form_feed",
 '\\':"backward_slash",
 '^':"xor_operator"
}

In [0]:
# data_path = 'datascience-dataset/Raw_Data'

### 2. Web Data Common Data Creation

In [0]:
with open(os.path.join(data_path,'Web_Data_Common.json'), 'r') as f:
    data = json.load(f)

In [0]:
web_data_common = pd.DataFrame(data)

In [0]:
values_to_column_mapping = pd.read_excel(os.path.join(data_path,'Dependant_Objects','values_to_column_mapping.xlsx'))
values_to_column_mapping = values_to_column_mapping[values_to_column_mapping['dataset']=='web_data_common']

In [0]:
merged_df = pd.merge(web_data_common,values_to_column_mapping,left_on='id',right_on='uniq_id',how='inner',suffixes=('','_check'))
merged_df['column_name'] = merged_df['column_name_check']
merged_df['value'] = merged_df.apply(lambda x: [val for val in x['value'] if x['column_name']!=val and len(val)>0],axis=1)

In [0]:
non_merged_df = web_data_common[~web_data_common['id'].isin(merged_df['uniq_id'])]
non_merged_df['value'] = non_merged_df.apply(lambda x: [val for val in x['value'] if x['column_name']!=val and len(val)>0],axis=1)

In [0]:
merged_df_sub = merged_df[non_merged_df.columns]
combined_df = pd.concat([merged_df_sub,non_merged_df])

In [0]:
assert combined_df.shape[0] == web_data_common.shape[0]

In [0]:
combined_df['column_name'] = combined_df.apply(lambda x: ''.join([char for char in x['column_name'] if ord(char)<128]),axis=1)
combined_df['column_name'] = combined_df.apply(lambda x: 'unknown_' +str(x['id']) if x['column_name'].strip(string.punctuation)=='' else x['column_name'],axis=1)
combined_df['column_name'] = combined_df.apply(lambda x:x['column_name'].replace('Imputed_','unknown_'),axis=1)

In [0]:
combined_df['column_name'] = combined_df['column_name'].str.strip()
# combined_df = combined_df[combined_df['value'].str.len()>0]
combined_df = combined_df.reset_index(drop=True)

#### 2.1. Duplication of data at type and column level: Aggregated

In [0]:
data_grouped = combined_df.groupby(['type','column_name'])['value'].apply(lambda x: [sub for ele in list(x) for sub in ele]).reset_index()
id_grouped = combined_df.groupby(['type','column_name'])['id'].first().reset_index()
combined_grouped = pd.merge(data_grouped,id_grouped,on=['type','column_name'])

In [0]:
assert combined_grouped.shape[0] == data_grouped.shape[0]
assert combined_grouped.shape[0] == id_grouped.shape[0]

In [0]:
combined_grouped['table_name'] = combined_grouped.apply(lambda x:x['type'].replace('.json',''),axis=1)
combined_grouped['dataset_name'] = 'web_data_common'

In [0]:
combined_grouped['column_name'] = combined_grouped['column_name'].str.strip()
combined_grouped['table_name'] = combined_grouped['table_name'].str.strip()
combined_grouped['dataset_name'] = combined_grouped['dataset_name'].str.strip()

In [0]:
combined_grouped['master_id'] = combined_grouped.apply(lambda x: x['dataset_name']+'$$##$$'+x['table_name']+'$$##$$'+x['column_name'],axis=1)
combined_grouped = combined_grouped.rename(columns={'value':'values'})
combined_grouped = combined_grouped.sort_values(by='id').reset_index(drop=True)

In [0]:
assert combined_grouped.shape[0]==combined_grouped['id'].nunique()
assert combined_grouped['master_id'].nunique()==combined_grouped.shape[0]

In [0]:
assert combined_grouped[combined_grouped['dataset_name']==''].shape[0] == 0
assert combined_grouped[combined_grouped['table_name']==''].shape[0] == 0
assert combined_grouped[combined_grouped['column_name']==''].shape[0] == 0

In [0]:
assert list(set([type(val) for val in combined_grouped['values'].tolist()]))[0] == list

In [0]:
combined_grouped = combined_grouped[['id','dataset_name','table_name','column_name','values']]

In [0]:
combined_grouped.to_json(os.path.join(data_path,'Source_Data','web_data_common','web_data_common.json'),orient='records')

In [0]:
string.punctuation

### 3. Kaggle+SQL Dump Data Correction

In [0]:
with open(os.path.join(data_path,'Kaggle_SQL_Assessment_Data.json'), 'r') as f:
    data = json.load(f)

In [0]:
kaggle_data = pd.DataFrame(data)
kaggle_data_sub = kaggle_data[['id','dataset_name','table_name','column_name','values']]

In [0]:
kaggle_data_sub['column_name'] = kaggle_data_sub.apply(lambda x: ''.join([char for char in x['column_name'] if ord(char)<128]),axis=1)
kaggle_data_sub['column_name'] = kaggle_data_sub.apply(lambda x: 'unknown_' +str(x['id']) if x['column_name'].strip(string.punctuation)=='' else x['column_name'],axis=1)
kaggle_data_sub['column_name'] = kaggle_data_sub.apply(lambda x:x['column_name'].replace('Imputed_','unknown_'),axis=1)

In [0]:
assert sum(kaggle_data_sub.groupby(['dataset_name','table_name','column_name'])['id'].count()>1) == 0

In [0]:
kaggle_data_sub = kaggle_data_sub[kaggle_data_sub['values'].str.len()>0]

In [0]:
kaggle_data_sub['column_name'] = kaggle_data_sub['column_name'].str.strip()
kaggle_data_sub['table_name'] = kaggle_data_sub['table_name'].str.strip()
kaggle_data_sub['dataset_name'] = kaggle_data_sub['dataset_name'].str.strip()

In [0]:
kaggle_data_sub['master_id'] = kaggle_data_sub.apply(lambda x: x['dataset_name']+'$$##$$'+x['table_name']+'$$##$$'+x['column_name'],axis=1)

In [0]:
assert kaggle_data_sub['master_id'].nunique()==kaggle_data_sub.shape[0]

In [0]:
kaggle_data_sub = kaggle_data_sub[['id','dataset_name','table_name','column_name','values']]

In [0]:
assert kaggle_data_sub[kaggle_data_sub['dataset_name']==''].shape[0] == 0
assert kaggle_data_sub[kaggle_data_sub['table_name']==''].shape[0] == 0
assert kaggle_data_sub[kaggle_data_sub['column_name']==''].shape[0] == 0

In [0]:
assert list(set([type(val) for val in kaggle_data_sub['values'].tolist()]))[0] == list

In [0]:
kaggle_data_sub.to_json(os.path.join(data_path,'Source_Data','swastik','kaggle_sql_data.json'),orient='records')

### 4. Sherlock Dataset

In [0]:
with open(os.path.join(data_path,'sherlock_data.json'), 'r') as f:
    data = json.load(f)

In [0]:
sherlock_data = pd.DataFrame(data)

In [0]:
updated_values = []
ignore_ids =[] 
for row,data in sherlock_data.iterrows():
    values = data['values']
    try:
        if isinstance(values,str):
            updated_values.append(ast.literal_eval(values))
        else:
            updated_values.append(values)
    except Exception as e:
        ignore_ids.append(data['id'])

In [0]:
len(ignore_ids)

Out[7]: 62

In [0]:
sherlock_data_sub = sherlock_data[~sherlock_data['id'].isin(ignore_ids)]

In [0]:
addl_sherlock_data = pd.read_excel(os.path.join(data_path,"Addl_Sherlock_data.xlsx"))
addl_sherlock_data['column_name'] = addl_sherlock_data['type']

In [0]:
req_cols = ['id','values','column_name','type']
final_sherlock_data = pd.concat([sherlock_data_sub[req_cols],addl_sherlock_data[req_cols]])

In [0]:
final_sherlock_data['values'] = final_sherlock_data.apply(lambda x: ast.literal_eval(x['values']) if isinstance(x['values'],str) else x['values'],axis=1)

In [0]:
values_to_column_mapping = pd.read_excel(os.path.join(data_path,'supporting_files','values_to_column_mapping.xlsx'))
values_to_column_mapping = values_to_column_mapping[values_to_column_mapping['dataset']=='sherlock']

In [0]:
final_sherlock_data['table_name'] ='sherlock_table'
final_sherlock_data['dataset_name'] = 'sherlock'

In [0]:
final_sherlock_data['column_name'] = final_sherlock_data['column_name'].str.strip()
final_sherlock_data['table_name'] = final_sherlock_data['table_name'].str.strip()
final_sherlock_data['dataset_name'] = final_sherlock_data['dataset_name'].str.strip()

In [0]:
merged_df = pd.merge(final_sherlock_data,values_to_column_mapping,left_on='id',right_on='uniq_id',how='inner',suffixes=('','_check'))
merged_df['column_name'] = merged_df['column_name_check']
merged_df['values'] = merged_df.apply(lambda x: [val for val in x['values'] if x['column_name']!=val and len(str(val))>0],axis=1)

In [0]:
non_merged_df = final_sherlock_data[~final_sherlock_data['id'].isin(merged_df['uniq_id'])]
non_merged_df['values'] = non_merged_df.apply(lambda x: [val for val in x['values'] if x['column_name']!=val and len(str(val))>0],axis=1)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_4928\2506999123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_merged_df['values'] = non_merged_df.apply(lambda x: [val for val in x['values'] if x['column_name']!=val and len(str(val))>0],axis=1)


In [0]:
merged_df_sub = merged_df[non_merged_df.columns]
combined_df = pd.concat([merged_df_sub,non_merged_df])

In [0]:
combined_df['column_name'] = combined_df.apply(lambda x:x['column_name'].replace('Imputed_','unknown_'),axis=1)

In [0]:
combined_df['column_name'] = combined_df['column_name'].str.strip()
combined_df['column_name'] = combined_df.apply(lambda x: x['type'] if x['column_name']=='' else x['column_name'],axis=1)
combined_df = combined_df[combined_df['values'].str.len()>0]
combined_df = combined_df.reset_index(drop=True)

In [0]:
# combined_df['column_increment'] = combined_df.groupby(['column_name'])["id"].rank(method="dense", ascending=False).astype(int)

In [0]:
combined_df['column_name'] = combined_df['column_name'].str.strip()
combined_df['column_name'] = combined_df.apply(lambda x: x['column_name']+'_'+str(x['id']),axis=1)

combined_df['table_name'] = combined_df['table_name'].str.strip()
combined_df['dataset_name'] = combined_df['dataset_name'].str.strip()

In [0]:
combined_df['master_id'] = combined_df.apply(lambda x: x['dataset_name'].replace('.json','')+'$$##$$'+x['table_name']+'$$##$$'+x['column_name'],axis=1)
# combined_df = combined_df.sort_values(by='id').reset_index(drop=True)

In [0]:
assert combined_df.shape[0]==combined_df['master_id'].nunique()

In [0]:
assert combined_df[combined_df['dataset_name']==''].shape[0] == 0
assert combined_df[combined_df['table_name']==''].shape[0] == 0
assert combined_df[combined_df['column_name']==''].shape[0] == 0

In [0]:
combined_df = combined_df[['id','dataset_name','table_name','column_name','values']]

In [0]:
print(list(set([type(val) for val in combined_df['values'].tolist()]))[0])
assert list(set([type(val) for val in combined_df['values'].tolist()]))[0] == list

<class 'list'>


In [0]:
combined_df.to_json(os.path.join(data_path,'Source_Data','sherlock','sherlock_data.json'),orient='records')

#### Appendix

In [0]:
# import re

# col_values = ['2.0','1.20'] #(combined_grouped[combined_grouped['id']==562]['values'].tolist()[0])

# def count_pattern_in_cells(values: list, pat):
#     return [len(re.findall(pat, s)) for s in values]


# def count_pattern_in_cells_with_non_zero_count(values: list, pat):
#     cell_counts = [len(re.findall(pat, s)) for s in values]

#     return sum(1 for c in cell_counts if c > 0), cell_counts

# NUMBER_PATTERN = re.compile(r"[0-9]")
# TEXT_PATTERN = re.compile(r"[a-zA-Z]")
# WORD_PATTERN = re.compile(r"[\w+]")
# SPECIAL_CHARACTERS_PATTERN = re.compile(r'[!#&\'()*+-/:;<=>?@[\\]^_`{|}~]')

# n_val = len(col_values)

# spec_char_counts = count_pattern_in_cells(col_values, SPECIAL_CHARACTERS_PATTERN)

# numeric_cell_nz_count, numeric_char_counts = count_pattern_in_cells_with_non_zero_count(col_values, NUMBER_PATTERN)
# text_cell_nz_count, text_char_counts = count_pattern_in_cells_with_non_zero_count(col_values, TEXT_PATTERN)

# alphanum_cell_counts = [len(col_values[idx]) if (numeric_char_counts[idx]>0 and (spec_char_counts[idx]>0 or text_char_counts[idx]>0)) else 0 for idx in range(len(col_values))]
# alphanum_cell_nz_count, alphanum_char_counts = sum(1 for c in alphanum_cell_counts if c > 0), alphanum_cell_counts

# print(text_cell_nz_count,numeric_cell_nz_count,spec_char_counts,alphanum_cell_counts,)
# print(alphanum_cell_nz_count / n_val if n_val > 0 else 0)

#####  Additional dataset

In [0]:
# train_parquet = pd.read_parquet(r"C:\Users\LENOVO\Documents\Sherlock\Data\Sherlock_Project\raw\train_values.parquet")

In [0]:
# train_labels = pd.read_parquet(r"C:/Users/LENOVO/Documents/Sherlock/Data/Sherlock_Project/raw/train_labels.parquet")
# train_labels = train_labels.reset_index()

In [0]:
# train_parquet['values'] = train_parquet.apply(lambda x: ast.literal_eval(x['values']),axis=1)
# train_parquet['len']= train_parquet.apply(lambda x: len(set(x['values'])),axis=1)

In [0]:
# one_value_data = train_parquet[train_parquet['len']==1]
# one_value_data = one_value_data.reset_index()
# merged_data = pd.merge(one_value_data[['index','values']],train_labels,on='index')
# # merged_data['type'].value_counts().reset_index().to_clipboard()

In [0]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# merged_data['le_encoded'] = le.fit_transform(merged_data['type'])
# train,test = train_test_split(merged_data,test_size=0.05,stratify=merged_data['le_encoded'])

In [0]:
# col_values = ['san','sa719','sa72$#']
# char_set = set(''.join(col_values))

# for c in string.printable:
#     if c in character_mapping.keys():
#         value_feature_name = f'n_[{character_mapping[c]}]'
#     else:
#         value_feature_name = f'n_[{c}]'
        
#     if c in char_set:
#         counts = [s.count(c) for s in col_values]
#         print(value_feature_name,counts)